In [33]:
import ROOT
from ROOT import gStyle
import numpy as np
import ctypes
import os
import pandas as pd

def biner(edges,bin_widths,histogram):
    if (len(edges)+1!=len(bin_widths)):
        print("Check edges and bin widths array sizes!")
        return
    
    bins=[]
    first_bin = histogram.GetXaxis().GetBinLowEdge(1)
    last_bin = histogram.GetXaxis().GetBinUpEdge(histogram.GetNbinsX())
    for i in range(0,len(edges)):
        n_spaces = int((edges[i] - first_bin)/bin_widths[i])
        bins = np.concatenate((bins,np.linspace(first_bin,edges[i],n_spaces,endpoint=False)))
        first_bin = edges[i]
        if edges[i]==edges[-1]:
            n_spaces = int((last_bin - edges[i])/bin_widths[i+1])
            bins = np.concatenate((bins,np.linspace(edges[i],last_bin,n_spaces,endpoint=False)))
            bins = np.concatenate((bins,[last_bin]))
    return bins

def normalization(hist_list,norm_bin):
    for hist in hist_list:
        for i in range(1,hist.GetNbinsX()+1):
            value=hist.GetBinContent(i)
            error=hist.GetBinError(i)
            sf=hist.GetBinWidth(i)/norm_bin
            hist.SetBinContent(i,value/sf)
            hist.SetBinError(i,error/sf)
numpy.set_printoptions(threshold=sys.maxsize)



def Unpack_Histo(histogram_file):
    bin_content = []
    bin_content_uncer =[]
    for i in range(1,histogram_file.GetNbinsX()+1):
        bin_content.append(histogram_file.GetBinContent(i))
        bin_content_uncer.append(histogram_file.GetBinError(i))
        
    return np.array(bin_content),np.array(bin_content_uncer)

############# CONFIGURING THE JOB ###########################
samples_path = "/home/diegol/HEP/VBF-Analysis/Zee/Plots/RecursiveFits/SR/"
sample1_name = "Zee_SherpaRW"
sample2_name = "Zee_MGRW"

qcd_1=1.0
qcd_2=1.0

fs_name = "Zee"

histos_no_rebin={
"n_bjets":[],
"lepiso":[],
"n_jets_interval":[],
"delta_R_leplep_basic_dphi_drap_btag_iso_pt1_pt2_j1pt_j2pt_ptbal_mjj_nji_zcen_mass_ptl":[],
"delta_R_lep1jet_basic_dphi_drap_btag_iso_pt1_pt2_j1pt_j2pt_ptbal_mjj_nji_zcen_mass_ptl":[],
"delta_R_lep2jet_basic_dphi_drap_btag_iso_pt1_pt2_j1pt_j2pt_ptbal_mjj_nji_zcen_mass_ptl":[],    
}
histos_rebin={
"delta_phi":[[2.0],[0.2,0.8],0.2],
"lep1_eta_basic_dphi_drap_btag_iso_pt1_pt2_j1pt_j2pt_ptbal_mjj_nji_zcen_mass_ptl":[[0.1],[0.2,0.199],0.2],
"lep2_eta_basic_dphi_drap_btag_iso_pt1_pt2_j1pt_j2pt_ptbal_mjj_nji_zcen_mass_ptl":[[0.1],[0.2,0.199],0.2],
"ljet0_eta_basic_cuts_ptl":[[-3.0,3.0],[0.5,0.2,0.5],0.2],
"ljet1_eta_basic_cuts_ptl":[[-3.0,3.0],[0.5,0.2,0.5],0.2],
"lep1_pt":[[300],[20,50],20],
"lep2_pt":[[300],[20,50],20],
"ljet0_pt":[[85,460],[85,15,54],15],
"ljet1_pt":[[80,440],[80,15,56],15],
"ljet2_pt_basic_cuts_ptl":[[100],[20,50],20],
"pt_bal":[[0.15,0.3],[0.03,0.05,0.7],0.15],
"Z_centrality":[[0.5],[0.1,0.5],0.1],
"delta_y":[[2.0,6.0],[1.0,0.5,1.0],1.0],
"inv_mass":[[70,110,140],[10,5,10,20],5],
"mass_jj":[[1500],[250,500],250],
"Z_pt_reco_basic_cuts_ptl":[[300],[20,50],20],
"vec_sum_pt_jets_basic_cuts_ptl":[[300],[20,50],20],
"ratio_zpt_sumjetpt_basic_cuts_ptl":[[0.75,1.25],[0.25,0.1,0.25],0.1],
"met_basic_dphi_drap_btag_iso_pt1_pt2_j1pt_j2pt_ptbal_mjj_nji_zcen_mass_ptl":[[50],[10,50],10],
}
total_histos ={**histos_no_rebin,**histos_rebin}

In [34]:
r.TH1.AddDirectory(r.kFALSE)
file1 = r.TFile.Open(samples_path+sample1_name+'.root')
file2 = r.TFile.Open(samples_path+sample2_name+'.root')

myFile =r.TFile.Open(fs_name+"_Average.root", "RECREATE")
r.TH1.AddDirectory(r.kFALSE)    

for histo in total_histos:

    h1 = file1.Get(histo)
    h2 = file2.Get(histo)
    
    if histo in histos_rebin:
    
        rebining=biner(total_histos[histo][0],total_histos[histo][1],h1)
        nb=len(rebining)-1
        h1=h1.Rebin(nb,histo,rebining)
        h2=h2.Rebin(nb,histo,rebining)
        
        h1.Scale(qcd_1)
        h2.Scale(qcd_2)

        hist_list=[h1,h2]
        normalization(hist_list,total_histos[histo][2])
    
    
    final_hist = h1.Clone()
    final_hist.Add(h2)
    final_hist.Scale(0.5)

    myFile.WriteObject(final_hist,histo)

myFile.Close()

os.system("mv "+fs_name+"_Average.root "+samples_path+"/"+fs_name+"_Average.root")

0

Warning in <TH1F::Rebin>: Bin edge 7 of rebinned histogram does not match any bin edges of the old histogram. Result can be inconsistent
Warning in <TH1F::Rebin>: Bin edge 7 of rebinned histogram does not match any bin edges of the old histogram. Result can be inconsistent


In [25]:

print(content1)


[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 3.61306357e+00 1.81559181e+01 3.35912094e+01 4.88206291e+01
 5.46771164e+01 4.77617989e+01 4.04087029e+01 4.09236145e+01
 3.25116577e+01 2.91105213e+01 2.37594490e+01 1.90360851e+01
 2.12926941e+01 1.49669847e+01 1.65707626e+01 1.35473185e+01
 8.70192432e+00 9.95579052e+00 8.03633022e+00 5.93532133e+00
 4.86358976e+00 4.23136234e+00 3.97852254e+00 2.35694408e+00
 2.31049681e+00 2.14221525e+00 1.75707197e+00 2.23770905e+00
 1.86860943e+00 2.40016317e+00 1.03567159e+00 8.33868861e-01
 8.04032087e-01 7.08849490e-01 5.33422589e-01 5.09694874e-01
 5.81120193e-01 3.96917284e-01 4.56649303e-01 2.56646663e-01
 3.14318001e-01 1.82123277e-02 1.81234539e-01 9.43045765e-02
 1.25293836e-01 1.53622717e-01 7.36540034e-02 1.58055425e-01
 1.02223627e-01 3.75888683e-02 9.86025333e-02 2.45991163e-02
 1.72666311e-02 2.15110909e-02 4.42183651e-02 8.87200050e-03]
